In [189]:
import numpy as np
import pandas as pd
import itertools
import warnings
warnings.filterwarnings('ignore')
from stoch_order import *
from scipy.stats import norm,multivariate_normal

In [2]:
%load_ext rpy2.ipython

# Categorical ordered data simulation

In [171]:
%%R -o ord,ord1,ord2
#install.packages("GenOrd")
require(GenOrd)

marginal <- list(c(0.1,0.3,0.6), c(0.4,0.7,0.9))
marginal1 <- list(c(0.6,0.85,0.9), c(0.05,0.2,0.9))
marginal2 <- list(c(0.2,0.3,0.6), c(0.05,0.2,0.8))

Sigma <- matrix(c(1,0.6,0.6,1),2,2)
Sigma1 <- matrix(c(1,0.6,0.6,1),2,2)
Sigma2 <- matrix(c(1,0.6,0.6,1),2,2)


ord = ordsample(100, marginal, Sigma)
ord1 = ordsample(100, marginal1, Sigma1)
ord2 = ordsample(100, marginal1, Sigma1)

In [172]:
ord = np.array(ord)
ord1 = np.array(ord1)
ord2 = np.array(ord2)

# Univariate Non-normal data simulation (Fleishman's method)

In [173]:
%%R -o nndata,nndata1,nndata2
#install.packages("SimMultiCorrData")
require(SimMultiCorrData)
nndata <- nonnormvar1(method = "Fleishman", means = 0, vars = 1,
  skews = 1, skurts = 1, cstart = NULL, n = 100, seed = 1234)
nndata1 <- nonnormvar1(method = "Fleishman", means = 10, vars = 1,
  skews = 0.6, skurts = 1, cstart = NULL, n = 100, seed = 1234)
nndata2 <- nonnormvar1(method = "Fleishman", means = 10, vars = 1,
  skews = 0.6, skurts = 1, cstart = NULL, n = 100, seed = 1234)

In [174]:
nnd = np.array(nndata.__getitem__(1)).reshape((100,1))
nnd1 = np.array(nndata1.__getitem__(1)).reshape((100,1))
nnd2 = np.array(nndata2.__getitem__(1)).reshape((100,1))

# Multivariate Non-normal data simulation (extension of Fleishman's method)

In [185]:
%%R -o nnmvdata,nnmvdata1,nnmvdata2

require(semTools)
Sigma <- matrix(0.60,4,4)
Sigma1 <- matrix(0.10,4,4)
Sigma2 <- matrix(0.10,4,4)

diag(Sigma) <- 1
diag(Sigma1) <-1
diag(Sigma2) <-1

nnmvdata <- mvrnonnorm(100, rep(0,4), Sigma, skewness = 1, kurtosis = 1,empirical = FALSE)
nnmvdata1 <- mvrnonnorm(100, rep(0.5,4), Sigma1, skewness = 1, kurtosis = 1,empirical = FALSE)
nnmvdata2 <- mvrnonnorm(100, rep(0.5,4), Sigma2, skewness = 1, kurtosis = 1,empirical = FALSE)

In [186]:
nnmvd = np.array(nnmvdata)
nnmvd1 = np.array(nnmvdata1)
nnmvd2 = np.array(nnmvdata2)

# Multivariate Normal data simulation


In [177]:
ndata = multivariate_normal.rvs(mean=[0,0], cov=np.array([1,0.7,0.7,1]).reshape(2,2), size=100, random_state=None)
ndata1 = multivariate_normal.rvs(mean=[0.0,0.0], cov=np.array([1,0.7,0.7,1]).reshape(2,2), size=100, random_state=None)
ndata2 = multivariate_normal.rvs(mean=[0.5,0.5], cov=np.array([1,0.5,0.5,1]).reshape(2,2), size=100, random_state=None)

# Method application

### Categorical ordered

In [178]:
data = np.zeros(300*4).reshape(300,4)
data[:,0] = np.arange(1,301)
data[:,1] = np.append(np.repeat(1,100),np.append(np.repeat(2,100),np.repeat(3,100),axis=0),axis=0)
data[:,2:] = np.vstack((ord,ord1,ord2))
data = pd.DataFrame(data)
data.columns = ["id","Treatment","Outcome1","Outcome2"]


pvalues = stochastic_ordering(data,100)
print(pvalues.get("gpvariables"))
print(pvalues.get("gpsplits"))
print(pvalues.get("ppvariables"))
print(pvalues.get("ppsplits"))


0.06
0.04
[1.         0.00990099]
[0.03960396 0.0990099 ]


### Univariate non-normal

In [181]:
data = np.zeros(300*3).reshape(300,3)
data[:,0] = np.arange(1,301)
data[:,1] = np.append(np.repeat(1,100),np.append(np.repeat(2,100),np.repeat(3,100),axis=0),axis=0)
data[:,2:] = np.vstack((nnd,nnd1,nnd2))
data = pd.DataFrame(data)
data.columns = ["id","Treatment","Outcome1"]

pvalues = stochastic_ordering(data,100)
print(pvalues.get("gpvariables"))
print(pvalues.get("gpsplits"))
print(pvalues.get("ppvariables"))
print(pvalues.get("ppsplits"))


0.0
0.0
[0.00990099]
[0.00990099 0.00990099]


### Multivariate non-normal

In [187]:
data = np.zeros(300*6).reshape(300,6)
data[:,0] = np.arange(1,301)
data[:,1] = np.append(np.repeat(1,100),np.append(np.repeat(2,100),np.repeat(3,100),axis=0),axis=0)
data[:,2:] = np.vstack((nnmvd,nnmvd1,nnmvd2))
data = pd.DataFrame(data)
data.columns = ["id","Treatment","Outcome1","Outcome2","Outcome3","Outcome4"]


pvalues = stochastic_ordering(data,100)
print(pvalues.get("gpvariables"))
print(pvalues.get("gpsplits"))
print(pvalues.get("ppvariables"))
print(pvalues.get("ppsplits"))

0.0
0.0
[0.00990099 0.00990099 0.00990099 0.01980198]
[0.00990099 0.00990099]


### Multivariate normal

In [191]:
data = np.zeros(300*4).reshape(300,4)
data[:,0] = np.arange(1,301)
data[:,1] = np.append(np.repeat(1,100),np.append(np.repeat(2,100),np.repeat(3,100),axis=0),axis=0)
data[:,2:] = np.vstack((ndata,ndata1,ndata2))
data = pd.DataFrame(data)
data.columns = ["id","Treatment","Outcome1","Outcome2"]


pvalues = stochastic_ordering(data,100)
print(pvalues.get("gpvariables"))
print(pvalues.get("gpsplits"))
print(pvalues.get("ppvariables"))
print(pvalues.get("ppsplits"))

0.0
0.0
[0.00990099 0.01980198]
[0.01980198 0.00990099]
